### Imports

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# 

### Model / data parameters

In [2]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

# 

### Train the model

In [4]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 37s 89ms/step - loss: 0.3698 - accuracy: 0.8878 - val_loss: 0.0804 - val_accuracy: 0.9780
Epoch 2/15
422/422 [==============================] - 37s 88ms/step - loss: 0.1095 - accuracy: 0.9664 - val_loss: 0.0600 - val_accuracy: 0.9842
Epoch 3/15
422/422 [==============================] - 41s 96ms/step - loss: 0.0832 - accuracy: 0.9741 - val_loss: 0.0505 - val_accuracy: 0.9870
Epoch 4/15
422/422 [==============================] - 36s 86ms/step - loss: 0.0713 - accuracy: 0.9786 - val_loss: 0.0409 - val_accuracy: 0.9898
Epoch 5/15
422/422 [==============================] - 35s 82ms/step - loss: 0.0604 - accuracy: 0.9814 - val_loss: 0.0370 - val_accuracy: 0.9898
Epoch 6/15
422/422 [==============================] - 37s 87ms/step - loss: 0.0556 - accuracy: 0.9825 - val_loss: 0.0360 - val_accuracy: 0.9905
Epoch 7/15
422/422 [==============================] - 37s 87ms/step - loss: 0.0506 - accuracy: 0.9843 - val_loss: 0.0332 - val_accuracy:

# 

### Evaluate the trained model

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.025754714384675026
Test accuracy: 0.9916999936103821


# 

### Convert model to tflite and save it in C format

In [7]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /var/folders/l2/590l7mh12zs5c8p1xj5r8h8h0000gn/T/tmp2jnoc08s/assets


In [8]:
from tensorflow.lite.python.util import convert_bytes_to_c_source


source_text, header_text = convert_bytes_to_c_source(tflite_model,  "mnist_tflite")

with  open('inc/mnist_tflite.h',  'w')  as  file:
    file.write(header_text)

with  open('src/mnist_tflite.cc',  'w')  as  file:
    file.write(source_text)